In [1]:
import geopandas as gpd
import xarray as xr
import pandas as pd


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
gdf = gpd.read_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_p.shp")
# gdf = gdf.head(50)
# gdf = gpd.read_file(r"..\..\PT-FireProg_v2.1_L2_p_meteo_short.shp")

print(gdf)

c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


      fid             fname  year    id type             sdate  \
0       1  Gouveia_10082015  2015     1    z  2015-08-10 14:30   
1       1  Gouveia_10082015  2015     2    z  2015-08-11 12:30   
2       1  Gouveia_10082015  2015     3    z  2015-08-11 12:30   
3       1  Gouveia_10082015  2015     4    p  2015-08-10 14:30   
4       1  Gouveia_10082015  2015     5    p  2015-08-11 03:00   
...   ...               ...   ...   ...  ...               ...   
3350  164  Sabugal_16082025  2025  3363    p  2025-08-18 13:15   
3351  164  Sabugal_16082025  2025  3364    p  2025-08-19 03:30   
3352  164  Sabugal_16082025  2025  3365    p  2025-08-19 10:30   
3353  164  Sabugal_16082025  2025  3366    p                na   
3354  164  Sabugal_16082025  2025  3367    p                na   

                 edate      inidoy      enddoy source  zp_link  burn_perio  \
0                   na   -1.000000   -1.000000  fserv        1           1   
1                   na   -1.000000   -1.000000    s

In [4]:
for i, coluna in enumerate(gdf.columns, 1):
    print(f"{i:3d}. {coluna}")

  1. fid
  2. fname
  3. year
  4. id
  5. type
  6. sdate
  7. edate
  8. inidoy
  9. enddoy
 10. source
 11. zp_link
 12. burn_perio
 13. area
 14. growth_rat
 15. ros_i
 16. ros_p
 17. spdir_i
 18. spdir_p
 19. int_i
 20. int_p
 21. duration_i
 22. duration_p
 23. qc
 24. 1_3y_fir_p
 25. 3_8y_fir_p
 26. 8_ny_fir_p
 27. elev_av
 28. aspect_av
 29. landform
 30. fuel_model
 31. f_load_av
 32. land_use
 33. land_use_d
 34. geometry


In [5]:
gdf_wgs = gdf.to_crs("EPSG:4326")

gdf_wgs['sdate'] = pd.to_datetime(gdf_wgs['sdate'], errors='coerce')
gdf_wgs['edate'] = pd.to_datetime(gdf_wgs['edate'], errors='coerce')

C:\Users\dias3\AppData\Local\Temp\ipykernel_7948\2914767487.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gdf_wgs['edate'] = pd.to_datetime(gdf_wgs['edate'], errors='coerce')


In [6]:
print(gdf_wgs.columns)

Index(['fid', 'fname', 'year', 'id', 'type', 'sdate', 'edate', 'inidoy',
       'enddoy', 'source', 'zp_link', 'burn_perio', 'area', 'growth_rat',
       'ros_i', 'ros_p', 'spdir_i', 'spdir_p', 'int_i', 'int_p', 'duration_i',
       'duration_p', 'qc', '1_3y_fir_p', '3_8y_fir_p', '8_ny_fir_p', 'elev_av',
       'aspect_av', 'landform', 'fuel_model', 'f_load_av', 'land_use',
       'land_use_d', 'geometry'],
      dtype='object')


In [7]:
ds = xr.open_dataset(r"../../Data/Interim/Meteorological_data/ERA5_NetCDF/ERA5_meteo_SL_p.nc")
# ds = xr.open_dataset(r"../../Data/Interim/Meteorological_data/ERA5_NetCDF/ERA5_meteo_SL_c_short.nc")
ds_SL = ds.sel(valid_time=["2015-08-03T14:00:00", "2015-08-03T15:00:00", "2024-08-27T15:00:00"])
 

import dask.array as da

vals = ds_SL['VentIdx'].data  # dask array
uniques = da.unique(vals).compute()
print(uniques)


ds = ds.rio.write_crs("EPSG:4326")
print(ds)

[8654.04415131 9043.95866394 9082.94487    9557.83748627 9755.15937805
           nan]
<xarray.Dataset> Size: 3GB
Dimensions:      (valid_time: 2486, latitude: 61, longitude: 41)
Coordinates:
  * valid_time   (valid_time) datetime64[ns] 20kB 2015-08-03T14:00:00 ... 202...
  * latitude     (latitude) float32 244B 43.0 42.9 42.8 42.7 ... 37.2 37.1 37.0
  * longitude    (longitude) float32 164B -10.0 -9.9 -9.8 ... -6.2 -6.1 -6.0
    spatial_ref  int32 4B 0
Data variables: (12/97)
    gp_m2s2      (valid_time, latitude, longitude) float32 25MB ...
    CBH_m        (valid_time, latitude, longitude) float64 50MB ...
    HigCC_p      (valid_time, latitude, longitude) float64 50MB ...
    LowCC_p      (valid_time, latitude, longitude) float64 50MB ...
    MidCC_p      (valid_time, latitude, longitude) float64 50MB ...
    TotCC_p      (valid_time, latitude, longitude) float64 50MB ...
    ...           ...
    wdi_300      (valid_time, latitude, longitude) float32 25MB ...
    vwv_950      (va

In [8]:
print(ds.data_vars)

Data variables:
    gp_m2s2   (valid_time, latitude, longitude) float32 25MB ...
    CBH_m     (valid_time, latitude, longitude) float64 50MB ...
    HigCC_p   (valid_time, latitude, longitude) float64 50MB ...
    LowCC_p   (valid_time, latitude, longitude) float64 50MB ...
    MidCC_p   (valid_time, latitude, longitude) float64 50MB ...
    TotCC_p   (valid_time, latitude, longitude) float64 50MB ...
    BLH_m     (valid_time, latitude, longitude) float64 50MB ...
    Cape      (valid_time, latitude, longitude) float64 50MB ...
    Cin       (valid_time, latitude, longitude) float64 50MB ...
    sW_7      (valid_time, latitude, longitude) float64 50MB ...
    sW_28     (valid_time, latitude, longitude) float64 50MB ...
    sW_100    (valid_time, latitude, longitude) float64 50MB ...
    sW_289    (valid_time, latitude, longitude) float64 50MB ...
    DC_12h    (valid_time, latitude, longitude) float64 50MB ...
    FFMC_12h  (valid_time, latitude, longitude) float64 50MB ...
    FWI_1

In [9]:
import numpy as np
from shapely.geometry import Polygon

def create_centered_cell_polygons(lons, lats):
    # Calcula meia resolução nas duas direções
    dlon = (lons[1] - lons[0]) / 2
    dlat = (lats[1] - lats[0]) / 2
    
    cell_polygons = []

    for i, lat in enumerate(lats):
        for j, lon in enumerate(lons):
            # Cria polígono centrado no ponto
            poly = Polygon([
                (lon - dlon, lat - dlat),
                (lon + dlon, lat - dlat),
                (lon + dlon, lat + dlat),
                (lon - dlon, lat + dlat),
            ])
            
            cell_polygons.append((i, j, poly))

    return cell_polygons


# ✅ Usa os pontos do dataset
lons = ds.longitude.values
lats = ds.latitude.values

print("Criando polígonos centrados nos pontos...")
cell_polygons = create_centered_cell_polygons(lons, lats)
print(f"Total de polígonos criados: {len(cell_polygons)}")

# ✅ Verificar tipo
i, j, first_poly = cell_polygons[0]
print("Tipo:", type(first_poly))
print("Tem método intersection?", hasattr(first_poly, "intersection"))


Criando polígonos centrados nos pontos...
Total de polígonos criados: 2501
Tipo: <class 'shapely.geometry.polygon.Polygon'>
Tem método intersection? True


In [10]:
# ============================================================
# OTIMIZAÇÃO DE PROCESSAMENTO METEO x PROGRESSÃO DE INCÊNDIOS
# ============================================================

import numpy as np
import pandas as pd
from collections import Counter
from shapely.prepared import prep   # MAIS RÁPIDO QUE pol.intersection NAS VERIFICAÇÕES

# ------------------------------------------------------------
# 1. Função para pré-calcular interseções uma única vez
# ------------------------------------------------------------

def precompute_intersections(polygon, cell_polygons):
    """
    Calcula uma vez para cada polígono:
    - quais células intersectam
    - a área da interseção
    """
    polygon_prep = prep(polygon)  # acelera testa_interseção
    selected = []

    for y_idx, x_idx, cell_poly in cell_polygons:
        # Teste rápido: bounding box e prepared polygon
        if not polygon_prep.intersects(cell_poly):
            continue

        inter = polygon.intersection(cell_poly)
        if inter.is_empty:
            continue

        area = inter.area
        if area > 0:
            selected.append((y_idx, x_idx, area))

    return selected


# ------------------------------------------------------------
# 2. Média ponderada usando interseções pré-calculadas
# ------------------------------------------------------------

def calc_media_timeseries(da_time_slice, intersec_list, ignore_no_value):
    """
    Usa interseções pré-calculadas (muito mais rápido).
    """
    spatial_means = []

    for t in range(da_time_slice.valid_time.size):
        da_single = da_time_slice.isel(valid_time=t)
        values = da_single.values

        vals = []
        areas = []

        for y_idx, x_idx, area in intersec_list:
            v = values[y_idx, x_idx]

            # Aplica no-value padrão do dataset
            if np.isnan(v):
                continue
            if ignore_no_value and v in (-1, -999):
                continue

            vals.append(v)
            areas.append(area)

        if len(vals) == 0:
            spatial_means.append(np.nan)
            continue

        vals = np.array(vals)
        areas = np.array(areas)

        weights = areas / areas.sum()
        spatial_means.append(np.sum(vals * weights))

    valid = [m for m in spatial_means if not np.isnan(m)]
    return np.mean(valid) if valid else np.nan


# ------------------------------------------------------------
# 3. Moda por time series
# ------------------------------------------------------------

def calc_moda_timeseries(da_time_slice, intersec_list):
    spatial_modes = []

    for t in range(da_time_slice.valid_time.size):
        da_single = da_time_slice.isel(valid_time=t)
        values = da_single.values

        vals = []
        for y_idx, x_idx, area in intersec_list:
            v = values[y_idx, x_idx]
            if not np.isnan(v):
                vals.append(int(round(v)))  # garantir inteiro para moda

        if len(vals) == 0:
            spatial_modes.append(np.nan)
        else:
            spatial_modes.append(Counter(vals).most_common(1)[0][0])

    modos_validos = [m for m in spatial_modes if not np.isnan(m)]
    return Counter(modos_validos).most_common(1)[0][0] if modos_validos else np.nan


# ============================================================
# PROCESSAMENTO PRINCIPAL OTIMIZADO
# ============================================================

variaveis = list(ds.data_vars)
vars_com_no_value = ['Cin','CBH_m','LFC_hPa','CCL_hPa','EL_m',
                     'gT_s_9','gT_9_8','gT_8_7','gT_7_5','gT_5_3',
                     'wSv_9','wSv_7','wSv_5','wSdir_9','wSdir_7','wSdir_5']
vars_moda = ['Haines']

# Cria colunas vazias
for var in variaveis:
    gdf_wgs[var + "_av"] = np.nan

gdf_valid = gdf_wgs[
    (~gdf_wgs['sdate'].isna()) &
    (~gdf_wgs['edate'].isna())
].copy()

print(f"Iniciando: {len(gdf_valid)} polígonos válidos...")

for idx_count, idx in enumerate(gdf_valid.index):

    row = gdf_valid.loc[idx]
    polygon = row.geometry
    start = row['sdate'] - pd.Timedelta(minutes=1)
    end   = row['edate'] + pd.Timedelta(minutes=1)

    # progresso
    if idx_count % 10 == 0:
        print(f"{idx_count}/{len(gdf_valid)} - Polígono {row['id']}")

    try:
        # Pré-calcular interseções: MAIOR GANHO DE PERFORMANCE
        intersec_list = precompute_intersections(polygon, cell_polygons)

        if len(intersec_list) == 0:
            print(f"Sem interseções no polígono {row['id']}")
            continue

        ds_time_slice = ds.sel(valid_time=slice(start, end))
        if ds_time_slice.valid_time.size == 0:
            print(f"Sem dados no período para polígono {row['id']}")
            continue

        # Loop otimizado
        for var in variaveis:
            da = ds_time_slice[var]

            if var in vars_moda:
                av = calc_moda_timeseries(da, intersec_list)
            else:
                ignore_no = (var in vars_com_no_value)
                av = calc_media_timeseries(da, intersec_list, ignore_no)

            gdf_wgs.at[idx, var + "_av"] = av

    except Exception as e:
        print(f"Erro no polígono {row['id']}: {e}")
        for var in variaveis:
            gdf_wgs.at[idx, var + "_av"] = np.nan

print("✅ Concluído!")

Iniciando: 1715 polígonos válidos...
0/1715 - Polígono 4


c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

10/1715 - Polígono 26
20/1715 - Polígono 75
Erro no polígono 117: TopologyException: side location conflict at -8.1716695311720393 40.908183120526246. This can occur if the input geometry is invalid.
30/1715 - Polígono 121
40/1715 - Polígono 131
50/1715 - Polígono 164
60/1715 - Polígono 194
70/1715 - Polígono 234
80/1715 - Polígono 244
90/1715 - Polígono 274
100/1715 - Polígono 298
110/1715 - Polígono 319
120/1715 - Polígono 338
130/1715 - Polígono 374
140/1715 - Polígono 390
150/1715 - Polígono 422
160/1715 - Polígono 432
170/1715 - Polígono 449
180/1715 - Polígono 461
190/1715 - Polígono 471
200/1715 - Polígono 481
210/1715 - Polígono 491
220/1715 - Polígono 532
230/1715 - Polígono 569
240/1715 - Polígono 579
250/1715 - Polígono 590
260/1715 - Polígono 600
270/1715 - Polígono 610
280/1715 - Polígono 620
290/1715 - Polígono 630
300/1715 - Polígono 640
310/1715 - Polígono 650
320/1715 - Polígono 660
330/1715 - Polígono 670
340/1715 - Polígono 680
350/1715 - Polígono 691
360/1715 - Polí

In [11]:
for col in gdf_wgs.columns:
    print(col)

fid
fname
year
id
type
sdate
edate
inidoy
enddoy
source
zp_link
burn_perio
area
growth_rat
ros_i
ros_p
spdir_i
spdir_p
int_i
int_p
duration_i
duration_p
qc
1_3y_fir_p
3_8y_fir_p
8_ny_fir_p
elev_av
aspect_av
landform
fuel_model
f_load_av
land_use
land_use_d
geometry
gp_m2s2_av
CBH_m_av
HigCC_p_av
LowCC_p_av
MidCC_p_av
TotCC_p_av
BLH_m_av
Cape_av
Cin_av
sW_7_av
sW_28_av
sW_100_av
sW_289_av
DC_12h_av
FFMC_12h_av
FWI_12h_av
t_2m_C_av
d_2m_C_av
u10_kh_av
v10_kh_av
u100_kh_av
v100_kh_av
sP_hPa_av
wv10_kh_av
wdir10_av
wv_Fb_av
wdir_Fb_av
wv100_k_av
wdir100_av
rh_2m_av
VPD_Pa_av
dfmc_av
sW_1m_av
sW_3m_av
LCL_hPa_av
LCL_m_av
HDW_av
Haines_av
wSv_9_av
wSdir_9_av
wSv_7_av
wSdir_7_av
wSv_5_av
wSdir_5_av
wSv_1_av
wSdir_1_av
gT_s_9_av
gT_9_8_av
gT_8_7_av
gT_7_5_av
gT_5_3_av
CMLG_av
LFC_hPa_av
CCL_hPa_av
EL_m_av
VentIdx_av
LiftIdx_av
gp_950_av
gp_850_av
gp_700_av
gp_500_av
gp_300_av
rh_950_av
rh_850_av
rh_700_av
rh_500_av
rh_300_av
t_950_av
t_850_av
t_700_av
t_500_av
t_300_av
u_950_av
u_850_av
u_700_

In [12]:
# --- CONFIGURAÇÃO DOS NÍVEIS ---
levels_config = [
    {'level': '10',   'input_suffix': '_kh', 'output_suffix': '_kh', 'suffix': 'r'},
    {'level': '100',  'input_suffix': '_kh', 'output_suffix': '_k', 'suffix': 'r'},
    {'level': '_950', 'input_suffix': '',    'output_suffix': '', 'suffix': ''},
    {'level': '_850', 'input_suffix': '',    'output_suffix': '', 'suffix': ''},
    {'level': '_700', 'input_suffix': '',    'output_suffix': '', 'suffix': ''},
    {'level': '_500', 'input_suffix': '',    'output_suffix': '', 'suffix': ''},
    {'level': '_300', 'input_suffix': '',    'output_suffix': '', 'suffix': ''},
]


print("🔄 Iniciando processamento de ventos...")

# --- PROCESSAMENTO PARA CADA NÍVEL ---
for config in levels_config:
    level_name = config['level']
    input_suffix = config['input_suffix']
    output_suffix = config['output_suffix']
    suffix = config['suffix']
    
    # Nomes das colunas (somente _av agora)
    u_av_col = f"u{level_name}{input_suffix}_av"
    v_av_col = f"v{level_name}{input_suffix}_av"

    print(f"Processing {u_av_col} and {v_av_col}")
    
    # Verificar se colunas existem
    required_cols = [u_av_col, v_av_col]
    if not all(col in gdf_wgs.columns for col in required_cols):
        print(f"⚠️  Colunas faltando para nível {level_name}, pulando...")
        continue
    
    # Máscara de validação (simplificada)
    valid_mask = (
        (gdf_wgs[u_av_col] != "na") &
        (gdf_wgs[v_av_col] != "na")
    )
    
    # Extrair componentes
    u_av = gdf_wgs.loc[valid_mask, u_av_col].astype(float)
    v_av = gdf_wgs.loc[valid_mask, v_av_col].astype(float)
    
    # ✅ CALCULAR APENAS A MÉDIA — sem mx/mn
    gdf_wgs.loc[valid_mask, f'wv{level_name}{output_suffix}_av'] = np.sqrt(u_av**2 + v_av**2)
    gdf_wgs.loc[valid_mask, f'wdi{suffix}{level_name}_av'] = (270 - np.degrees(np.arctan2(v_av, u_av))) % 360

    print(f"Calculated {f'wv{level_name}{output_suffix}_av'} and {f'wdi{suffix}{level_name}_av'}")
    
    print(f"✅ Nível {level_name} processado: {valid_mask.sum()} registros válidos")

print("🎯 Processamento completo! Apenas colunas _av foram calculadas.")


🔄 Iniciando processamento de ventos...
Processing u10_kh_av and v10_kh_av
Calculated wv10_kh_av and wdir10_av
✅ Nível 10 processado: 3355 registros válidos
Processing u100_kh_av and v100_kh_av
Calculated wv100_k_av and wdir100_av
✅ Nível 100 processado: 3355 registros válidos
Processing u_950_av and v_950_av
Calculated wv_950_av and wdi_950_av
✅ Nível _950 processado: 3355 registros válidos
Processing u_850_av and v_850_av
Calculated wv_850_av and wdi_850_av
✅ Nível _850 processado: 3355 registros válidos
Processing u_700_av and v_700_av
Calculated wv_700_av and wdi_700_av
✅ Nível _700 processado: 3355 registros válidos
Processing u_500_av and v_500_av
Calculated wv_500_av and wdi_500_av
✅ Nível _500 processado: 3355 registros válidos
Processing u_300_av and v_300_av
Calculated wv_300_av and wdi_300_av
✅ Nível _300 processado: 3355 registros válidos
🎯 Processamento completo! Apenas colunas _av foram calculadas.


In [13]:
for col in gdf_wgs.columns:
    print(col)

fid
fname
year
id
type
sdate
edate
inidoy
enddoy
source
zp_link
burn_perio
area
growth_rat
ros_i
ros_p
spdir_i
spdir_p
int_i
int_p
duration_i
duration_p
qc
1_3y_fir_p
3_8y_fir_p
8_ny_fir_p
elev_av
aspect_av
landform
fuel_model
f_load_av
land_use
land_use_d
geometry
gp_m2s2_av
CBH_m_av
HigCC_p_av
LowCC_p_av
MidCC_p_av
TotCC_p_av
BLH_m_av
Cape_av
Cin_av
sW_7_av
sW_28_av
sW_100_av
sW_289_av
DC_12h_av
FFMC_12h_av
FWI_12h_av
t_2m_C_av
d_2m_C_av
u10_kh_av
v10_kh_av
u100_kh_av
v100_kh_av
sP_hPa_av
wv10_kh_av
wdir10_av
wv_Fb_av
wdir_Fb_av
wv100_k_av
wdir100_av
rh_2m_av
VPD_Pa_av
dfmc_av
sW_1m_av
sW_3m_av
LCL_hPa_av
LCL_m_av
HDW_av
Haines_av
wSv_9_av
wSdir_9_av
wSv_7_av
wSdir_7_av
wSv_5_av
wSdir_5_av
wSv_1_av
wSdir_1_av
gT_s_9_av
gT_9_8_av
gT_8_7_av
gT_7_5_av
gT_5_3_av
CMLG_av
LFC_hPa_av
CCL_hPa_av
EL_m_av
VentIdx_av
LiftIdx_av
gp_950_av
gp_850_av
gp_700_av
gp_500_av
gp_300_av
rh_950_av
rh_850_av
rh_700_av
rh_500_av
rh_300_av
t_950_av
t_850_av
t_700_av
t_500_av
t_300_av
u_950_av
u_850_av
u_700_

In [14]:
def calc_range_timeseries(da_time_slice, intersec_list, ignore_no_value):
    """
    Calcula a variação temporal (max − min) usando a média espacial
    ponderada pela área, exatamente como o cálculo principal.
    """
    spatial_vals = []

    for t in range(da_time_slice.valid_time.size):
        da_single = da_time_slice.isel(valid_time=t)
        values = da_single.values

        vals = []
        areas = []

        for y_idx, x_idx, area in intersec_list:
            v = values[y_idx, x_idx]
            if np.isnan(v):
                continue
            if ignore_no_value and v in (-1, -999):
                continue

            vals.append(v)
            areas.append(area)

        if len(vals) == 0:
            continue

        vals = np.array(vals)
        areas = np.array(areas)
        weights = areas / areas.sum()

        spatial_val = np.sum(vals * weights)
        spatial_vals.append(spatial_val)

    if len(spatial_vals) == 0:
        return np.nan

    return np.max(spatial_vals) - np.min(spatial_vals)


In [15]:
print("\nCalculando taxa temporal BLH_m_rt...")

# Criar coluna caso ainda não exista
if "BLH_m_rt" not in gdf_wgs.columns:
    gdf_wgs["BLH_m_rt"] = np.nan

for idx in gdf_valid.index:

    try:
        row = gdf_valid.loc[idx]
        polygon = row.geometry

        # Pré-calcular interseções
        intersec_list = precompute_intersections(polygon, cell_polygons)

        if len(intersec_list) == 0:
            gdf_wgs.at[idx, "BLH_m_rt"] = np.nan
            continue

        start = row['sdate'] - pd.Timedelta(minutes=1)
        end   = row['edate'] + pd.Timedelta(minutes=1)

        # slice temporal seguro
        da_time_slice = ds["BLH_m"].sel(valid_time=slice(start, end))
        if da_time_slice.valid_time.size < 2:  # menos de 2 instantes
            gdf_wgs.at[idx, "BLH_m_rt"] = np.nan
            continue

        # cálculo do valor espacial ponderado
        spatial_vals = []

        for t in range(da_time_slice.valid_time.size):
            da_single = da_time_slice.isel(valid_time=t)
            values = da_single.values

            vals = []
            areas = []

            for y_idx, x_idx, area in intersec_list:
                v = values[y_idx, x_idx]
                if np.isnan(v):
                    continue
                vals.append(v)
                areas.append(area)

            if len(vals) == 0:
                continue

            vals = np.array(vals)
            areas = np.array(areas)
            weights = areas / areas.sum()

            spatial_val = np.sum(vals * weights)
            spatial_vals.append(spatial_val)

        if len(spatial_vals) < 2:
            gdf_wgs.at[idx, "BLH_m_rt"] = np.nan
            continue

        # Determinar min e max e seus índices
        spatial_vals = np.array(spatial_vals)
        min_idx = spatial_vals.argmin()
        max_idx = spatial_vals.argmax()
        min_val = spatial_vals[min_idx]
        max_val = spatial_vals[max_idx]

        # Diferença de tempo entre os extremos
        times = da_time_slice.valid_time.values
        t_min = times[min_idx]
        t_max = times[max_idx]
        delta_hours = (t_max - t_min) / np.timedelta64(1, 'h')  # diferença em horas

        if delta_hours == 0:
            gdf_wgs.at[idx, "BLH_m_rt"] = np.nan
            continue

        # Calcular taxa
        if min_idx < max_idx:
            blh_rate = (max_val - min_val) / delta_hours  # subida
        else:
            blh_rate = (min_val - max_val) / delta_hours  # descida

        gdf_wgs.at[idx, "BLH_m_rt"] = blh_rate

    except Exception as e:
        print(f"Erro no polígono {row['id']} (BLH_m_rt): {e}")

print("✅ BLH_m_rt (taxa) calculada!")


Calculando taxa temporal BLH_m_rt...


c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


Erro no polígono 117 (BLH_m_rt): TopologyException: side location conflict at -8.1716695311720393 40.908183120526246. This can occur if the input geometry is invalid.
Erro no polígono 1106 (BLH_m_rt): TopologyException: side location conflict at -7.9790470582155324 39.894966074754478. This can occur if the input geometry is invalid.
Erro no polígono 2535 (BLH_m_rt): TopologyException: side location conflict at -8.6946505074250364 37.466348511921467. This can occur if the input geometry is invalid.
✅ BLH_m_rt (taxa) calculada!


In [16]:
print("\nCalculando Recirculation Index + Circular Variance...")

# Cria colunas caso ainda não existam
for col in ["Recirc", "CircVar"]:
    if col not in gdf_wgs.columns:
        gdf_wgs[col] = np.nan

T = 3600  # 1h em segundos

for idx in gdf_valid.index:
    try:
        row = gdf_valid.loc[idx]
        polygon = row.geometry

        # ✅ usa a função rápida já existente
        intersec_list = precompute_intersections(polygon, cell_polygons)

        if len(intersec_list) == 0:
            gdf_wgs.loc[idx, ["Recirc", "CircVar"]] = np.nan
            continue

        start = row["sdate"]
        end   = row["edate"]

        u_da = ds["u10_kh"].sel(valid_time=slice(start, end))
        v_da = ds["v10_kh"].sel(valid_time=slice(start, end))

        if u_da.valid_time.size == 0 or v_da.valid_time.size == 0:
            gdf_wgs.loc[idx, ["Recirc", "CircVar"]] = np.nan
            continue

        # ✅ extrai valores para TODAS as células intersectadas
        weights = np.array([area for (_, _, area) in intersec_list], dtype=float)
        weights /= weights.sum()   # normaliza

        u_ts = []
        v_ts = []

        # ✅ loop temporal
        for t in range(u_da.valid_time.size):
            u_slice = u_da.isel(valid_time=t).values
            v_slice = v_da.isel(valid_time=t).values

            u_vals = []
            v_vals = []

            # ✅ aplica os pesos às células intersectadas
            for (i, j, _) in intersec_list:
                u_vals.append(u_slice[i, j])
                v_vals.append(v_slice[i, j])

            u_vals = np.array(u_vals)
            v_vals = np.array(v_vals)

            # ignora NaNs estratégicos
            mask = (~np.isnan(u_vals)) & (~np.isnan(v_vals))
            if mask.sum() == 0:
                u_ts.append(np.nan)
                v_ts.append(np.nan)
            else:
                w_eff = weights[mask]
                w_eff /= w_eff.sum()

                u_ts.append(np.sum(u_vals[mask] * w_eff))
                v_ts.append(np.sum(v_vals[mask] * w_eff))

        u_ts = np.array(u_ts)
        v_ts = np.array(v_ts)

        # remove NaNs ao longo do tempo
        mask = (~np.isnan(u_ts)) & (~np.isnan(v_ts))
        if mask.sum() < 2:
            gdf_wgs.loc[idx, ["Recirc", "CircVar"]] = np.nan
            continue

        u_w = u_ts[mask]
        v_w = v_ts[mask]

        # ===== RECIRC (Russo et al.) =====
        L = T * np.sqrt((u_w.sum())**2 + (v_w.sum())**2)
        S = T * np.sum(np.sqrt(u_w**2 + v_w**2))
        Recirc = 1 - (L / S) if S > 0 else np.nan

        # ===== Estatística circular =====
        phi = np.arctan2(v_w, u_w)
        n = phi.size
        C = np.sum(np.cos(phi))
        S_ = np.sum(np.sin(phi))
        Rlen = np.sqrt(C**2 + S_**2) / n
        CircVar = 1 - Rlen

        # guarda
        gdf_wgs.loc[idx, "Recirc"] = Recirc
        gdf_wgs.loc[idx, "CircVar"] = CircVar

    except Exception as e:
        print(f"Erro no polígono {row['id']}: {e}")



Calculando Recirculation Index + Circular Variance...


c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\geopandas\geodataframe.py:1968: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


Erro no polígono 117: TopologyException: side location conflict at -8.1716695311720393 40.908183120526246. This can occur if the input geometry is invalid.
Erro no polígono 1106: TopologyException: side location conflict at -7.9790470582155324 39.894966074754478. This can occur if the input geometry is invalid.
Erro no polígono 2535: TopologyException: side location conflict at -8.6946505074250364 37.466348511921467. This can occur if the input geometry is invalid.


In [17]:

# Listas de variáveis por valor nulo especial

vars_com_no_value_1 = ['Cin_av']

vars_com_no_value_minus1 = ['CBH_m_av', 'LFC_hPa_av', 'CCL_hPa_av', 'EL_m_av', 'Recirc', 'CircVar', 'CircStd_dg']

vars_com_no_value_minus999 = [
    'BLH_m_rt', 'gT_s_9_av', 'gT_9_8_av', 'gT_8_7_av', 'gT_7_5_av', 'gT_5_3_av', 
    'wSv_9_av', 'wSdir_9_av', 'wSv_7_av', 'wSdir_7_av', 'wSv_5_av', 'wSdir_5_av', 
    'wSv_1_av', 'wSdir_1_av'
]

# Dicionário combinando todas as listas

no_value_map = {
    **{var: 1 for var in vars_com_no_value_1},
    **{var: -1 for var in vars_com_no_value_minus1},
    **{var: -999 for var in vars_com_no_value_minus999},
}

# Preencher os NaNs conforme o valor nulo definido

for var, novalue in no_value_map.items():
    if var in gdf_wgs.columns:
        mask = gdf_wgs["t_2m_C_av"].notna() & gdf_wgs[var].isna()
        n_filled = mask.sum()
        gdf_wgs.loc[mask, var] = novalue
        print(f"{var}: {n_filled} valores preenchidos com {novalue}")


Cin_av: 0 valores preenchidos com 1
CBH_m_av: 816 valores preenchidos com -1
LFC_hPa_av: 707 valores preenchidos com -1
CCL_hPa_av: 0 valores preenchidos com -1
EL_m_av: 1494 valores preenchidos com -1
Recirc: 138 valores preenchidos com -1
CircVar: 138 valores preenchidos com -1
BLH_m_rt: 138 valores preenchidos com -999
gT_s_9_av: 982 valores preenchidos com -999
gT_9_8_av: 0 valores preenchidos com -999
gT_8_7_av: 0 valores preenchidos com -999
gT_7_5_av: 0 valores preenchidos com -999
gT_5_3_av: 0 valores preenchidos com -999
wSv_9_av: 0 valores preenchidos com -999
wSdir_9_av: 0 valores preenchidos com -999
wSv_7_av: 0 valores preenchidos com -999
wSdir_7_av: 0 valores preenchidos com -999
wSv_5_av: 0 valores preenchidos com -999
wSdir_5_av: 0 valores preenchidos com -999
wSv_1_av: 0 valores preenchidos com -999
wSdir_1_av: 0 valores preenchidos com -999


In [18]:
print(gdf_wgs.columns)

Index(['fid', 'fname', 'year', 'id', 'type', 'sdate', 'edate', 'inidoy',
       'enddoy', 'source',
       ...
       'wdi_500_av', 'wdi_300_av', 'vwv_950_av', 'vwv_850_av', 'vwv_700_av',
       'vwv_500_av', 'vwv_300_av', 'BLH_m_rt', 'Recirc', 'CircVar'],
      dtype='object', length=134)


In [19]:
gdf_wgs = gdf_wgs.to_crs("EPSG:32629")

# Converte para datetime — erros viram NaT
gdf_wgs['sdate'] = pd.to_datetime(gdf_wgs['sdate'], errors='coerce')
gdf_wgs['edate'] = pd.to_datetime(gdf_wgs['edate'], errors='coerce')

# Agora podes usar .dt sem erro
gdf_wgs['sdate'] = gdf_wgs['sdate'].dt.strftime('%Y-%m-%d %H:%M:%S').fillna('na')
gdf_wgs['edate'] = gdf_wgs['edate'].dt.strftime('%Y-%m-%d %H:%M:%S').fillna('na')


In [20]:
gdf_wgs.drop(columns=[
    'u10_kh_av',
    'v10_kh_av',
    'u100_kh_av',
    'v100_kh_av',
    'v_950_av',
    'u_950_av',
    'u_850_av',
    'v_850_av',
    'u_700_av',
    'v_700_av',
    'u_500_av',
    'v_500_av',
    'u_300_av',
    'v_300_av',
], inplace=True, errors='ignore')


In [21]:
for vars in gdf_wgs.columns:
    print(vars)

fid
fname
year
id
type
sdate
edate
inidoy
enddoy
source
zp_link
burn_perio
area
growth_rat
ros_i
ros_p
spdir_i
spdir_p
int_i
int_p
duration_i
duration_p
qc
1_3y_fir_p
3_8y_fir_p
8_ny_fir_p
elev_av
aspect_av
landform
fuel_model
f_load_av
land_use
land_use_d
geometry
gp_m2s2_av
CBH_m_av
HigCC_p_av
LowCC_p_av
MidCC_p_av
TotCC_p_av
BLH_m_av
Cape_av
Cin_av
sW_7_av
sW_28_av
sW_100_av
sW_289_av
DC_12h_av
FFMC_12h_av
FWI_12h_av
t_2m_C_av
d_2m_C_av
sP_hPa_av
wv10_kh_av
wdir10_av
wv_Fb_av
wdir_Fb_av
wv100_k_av
wdir100_av
rh_2m_av
VPD_Pa_av
dfmc_av
sW_1m_av
sW_3m_av
LCL_hPa_av
LCL_m_av
HDW_av
Haines_av
wSv_9_av
wSdir_9_av
wSv_7_av
wSdir_7_av
wSv_5_av
wSdir_5_av
wSv_1_av
wSdir_1_av
gT_s_9_av
gT_9_8_av
gT_8_7_av
gT_7_5_av
gT_5_3_av
CMLG_av
LFC_hPa_av
CCL_hPa_av
EL_m_av
VentIdx_av
LiftIdx_av
gp_950_av
gp_850_av
gp_700_av
gp_500_av
gp_300_av
rh_950_av
rh_850_av
rh_700_av
rh_500_av
rh_300_av
t_950_av
t_850_av
t_700_av
t_500_av
t_300_av
wv_950_av
wv_850_av
wv_700_av
wv_500_av
wv_300_av
wdi_950_av
wdi_8

In [22]:
gdf_wgs.to_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_p_meteo.shp")
print("Shapefile salvo com médias meteorológicas")

print(gdf_wgs.head(50))

C:\Users\dias3\AppData\Local\Temp\ipykernel_7948\3432901670.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_wgs.to_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireProg_v2.1_L2_p_meteo.shp")
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'FFMC_12h_av' to 'FFMC_12h_a'
  ogr_write(


Shapefile salvo com médias meteorológicas
    fid                        fname  year  id type                sdate  \
0     1             Gouveia_10082015  2015   1    z  2015-08-10 14:30:00   
1     1             Gouveia_10082015  2015   2    z  2015-08-11 12:30:00   
2     1             Gouveia_10082015  2015   3    z  2015-08-11 12:30:00   
3     1             Gouveia_10082015  2015   4    p  2015-08-10 14:30:00   
4     1             Gouveia_10082015  2015   5    p  2015-08-11 03:00:00   
5     1             Gouveia_10082015  2015   6    p  2015-08-11 12:30:00   
6     1             Gouveia_10082015  2015   7    p  2015-08-11 12:30:00   
7     1             Gouveia_10082015  2015   8    p                   na   
8     1             Gouveia_10082015  2015   9    p                   na   
9     1             Gouveia_10082015  2015  10    p                   na   
10    2             Oleiros_03082015  2015  11    z  2015-08-03 14:30:00   
11    2             Oleiros_03082015  2015  12

In [23]:
df = gdf_wgs.drop(columns='geometry')
print(df)

      fid             fname  year    id type                sdate  \
0       1  Gouveia_10082015  2015     1    z  2015-08-10 14:30:00   
1       1  Gouveia_10082015  2015     2    z  2015-08-11 12:30:00   
2       1  Gouveia_10082015  2015     3    z  2015-08-11 12:30:00   
3       1  Gouveia_10082015  2015     4    p  2015-08-10 14:30:00   
4       1  Gouveia_10082015  2015     5    p  2015-08-11 03:00:00   
...   ...               ...   ...   ...  ...                  ...   
3350  164  Sabugal_16082025  2025  3363    p  2025-08-18 13:15:00   
3351  164  Sabugal_16082025  2025  3364    p  2025-08-19 03:30:00   
3352  164  Sabugal_16082025  2025  3365    p  2025-08-19 10:30:00   
3353  164  Sabugal_16082025  2025  3366    p                   na   
3354  164  Sabugal_16082025  2025  3367    p                   na   

                    edate      inidoy      enddoy source  zp_link  burn_perio  \
0                      na   -1.000000   -1.000000  fserv        1           1   
1        

In [24]:
ds.close()